In [231]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import redditutils


from string import punctuation

from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from gensim import corpora, models, similarities, matutils

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Suppress Pandas' automatic conversion of utc column to scientific notation:

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [43]:
df = pd.read_csv('showerthoughts.csv')

In [4]:
len(df[df.score > 1]) / len(df)

0.392702

39% of submissions get more than one upvote. This makes "greater than one upvote" a good candidate for our criteria for a good post.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
author          1000000 non-null object
title           1000000 non-null object
subreddit       1000000 non-null object
score           1000000 non-null int64
num_comments    1000000 non-null int64
retrieved_on    1000000 non-null int64
id              1000000 non-null object
created_utc     1000000 non-null int64
selftext        591622 non-null object
dtypes: int64(4), object(5)
memory usage: 68.7+ MB


In [44]:
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')

In [7]:
pd.set_option('display.max_colwidth', 999)

In [134]:
df.head()

,author,title,subreddit,score,num_comments,retrieved_on,id,created_utc,selftext,quality
0,os_coxae,"everyone that's ever said ""i'm speechless"" is a goddamn liar.",Showerthoughts,0,0,1466445837,4ixl2n,2016-05-11 22:38:48,NaN,0
1,Calrizle,i wonder how many people i've talked to who have been doing kaigles during our conversation.,Showerthoughts,0,0,1466494743,4lfjxn,2016-05-28 12:03:56,NaN,0
2,[deleted],"""palindrome"" is a let down as a word. rhinoplasty sure isn't.",Showerthoughts,0,0,1466493929,4ldt85,2016-05-28 01:21:03,[deleted],0
3,[deleted],"when you tell someone to keep a secret, it is no longer a secret",Showerthoughts,0,0,1466423373,4hmcbg,2016-05-03 08:30:35,[deleted],0
4,[deleted],"someone had the the time to create the words, ""dick"", ""penis"", and ""vagina""",Showerthoughts,0,0,1466474842,4k9ngz,2016-05-20 18:29:28,[deleted],0


In [135]:
df.sample()

,author,title,subreddit,score,num_comments,retrieved_on,id,created_utc,selftext,quality
990321,Jolkro,"tattoos and babies have the same tabu. no one says they are ugly to their owners, but you can be sure the ugliness is talked about to everyone else.",Showerthoughts,1077,49,1536660067,91qs2i,2018-07-25 10:51:13,[removed],1


In [137]:
X = df[['title', 'created_utc']]

In [136]:
y = df['quality']

In [158]:
train = df.iloc[:int(len(df)*.70), :]
test = df.iloc[int(len(df)*.70):, :]

## Clean the data

In [149]:
df_short = df.iloc[:100000, :].copy()

In [150]:
df_short['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')

In [151]:
df_short['title'] = df_short['title'].apply(lambda x: strip_punc(x))

In [152]:
df_short['title'] = df_short['title'].apply(lambda x: redditutils.lower_string(x))

In [153]:
df_short['title'] = df_short['title'].apply(lambda x: redditutils.strip_nums(x))

In [201]:
df_short['title'].apply(lambda x: strip_punc(x))

0                                                                                                                                                                     everyone thats ever said im speechless is a goddamn liar
1                                                                                                                                   i wonder how many people ive talked to who have been doing kaigles during our conversation
2                                                                                                                                                                     palindrome is a let down as a word rhinoplasty sure isnt
3                                                                                                                                                              when you tell someone to keep a secret it is no longer a secret
4                                                                                                           

In [157]:
df_short_train = df.iloc[:int(len(df_short)*.70), :]

In [283]:
df_short_test = df.iloc[int(len(df_short)*.70):int(len(df_short)), :]

In [159]:
df_short_test.head()

,author,title,subreddit,score,num_comments,retrieved_on,id,created_utc,selftext,quality
70000,AmagicFish,"if sony comes out with a k playstation, they should call it ""psk""",Showerthoughts,1,0,1463484931,4b8o1a,2016-03-20 18:34:50,[removed],0
70001,Phileas_Fogg,they guy that set up clinton's email server probably asked for help here on reddit.,Showerthoughts,1,0,1463496352,4bwp2r,2016-03-25 14:35:59,[removed],0
70002,Thenotoriouscanadian,"today, march th, is pi day, nap day and steak &amp; bj day",Showerthoughts,1,0,1463470751,4aevlu,2016-03-14 20:16:49,[removed],0
70003,theredwillow,"now that facebook shows users pages similar to what they've commented on before, the pages get more hateful comments",Showerthoughts,1,0,1463483543,4b5p18,2016-03-20 01:11:19,NaN,0
70004,[deleted],life hacks,Showerthoughts,1,0,1463457576,49n8my,2016-03-09 08:39:09,[deleted],0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import re

def my_tokenizer(doc, tkpat=re.compile('\\b[a-z][a-z]+\\b')):
    return [stemmer.stem(token) for token in tkpat.findall(doc)]

vec = CountVectorizer(tokenizer=my_tokenizer)

tf = vec.fit_transform(["hey running ran jumped","ok it worked"])

vec.vocabulary_

In [220]:
words_corpus = set(words.words())
analyzer = CountVectorizer().build_analyzer()
stem = SnowballStemmer('english')

def english_corpus(doc, stemmer=stem):
    return [stemmer.stem(w) for w in analyzer(doc) if w in words_corpus]

cv = CountVectorizer(stop_words='english', analyzer=english_corpus, min_df = 2, max_df = .95, ngram_range=(1, 2),
                     strip_accents='unicode', encoding='utf-8')

In [239]:
X_short_train = cv.fit_transform(df_short_train['title'])

In [285]:
X_short_test = cv.fit_transform(df_short_test['title'])

In [224]:
df_short_train.head()

,author,title,subreddit,score,num_comments,retrieved_on,id,created_utc,selftext,quality
0,os_coxae,"everyone that's ever said ""i'm speechless"" is a goddamn liar.",Showerthoughts,0,0,1466445837,4ixl2n,2016-05-11 22:38:48,NaN,0
1,Calrizle,i wonder how many people i've talked to who have been doing kaigles during our conversation.,Showerthoughts,0,0,1466494743,4lfjxn,2016-05-28 12:03:56,NaN,0
2,[deleted],"""palindrome"" is a let down as a word. rhinoplasty sure isn't.",Showerthoughts,0,0,1466493929,4ldt85,2016-05-28 01:21:03,[deleted],0
3,[deleted],"when you tell someone to keep a secret, it is no longer a secret",Showerthoughts,0,0,1466423373,4hmcbg,2016-05-03 08:30:35,[deleted],0
4,[deleted],"someone had the the time to create the words, ""dick"", ""penis"", and ""vagina""",Showerthoughts,0,0,1466474842,4k9ngz,2016-05-20 18:29:28,[deleted],0


In [225]:
counts = X_short_train.transpose()

In [226]:
counts.shape

(8798, 70000)

In [227]:
corpus = matutils.Sparse2Corpus(counts)

In [228]:
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [229]:
lda = models.LdaModel(corpus=corpus, num_topics=20, minimum_probability=.03, id2word=id2word, passes=10)

In [210]:
for x in [lda.log_perplexity]:
    print(x)

<bound method LdaModel.log_perplexity of <gensim.models.ldamodel.LdaModel object at 0x11f7daf98>>


In [215]:
df_short_train[df_short_train['title'].str.contains('infer')]

,author,title,subreddit,score,num_comments,retrieved_on,id,created_utc,selftext,quality
4585,pmf,vermin supreme should rename himself to vermin inferior (given recent developments),Showerthoughts,0,0,1466429549,4hzffd,2016-05-05 11:11:57,NaN,0
8090,[deleted],"by being oppressed and left out by men for millennia, women could have branched out evolutionarily with an inferior intellect, yet now they show the same mental capacity as men.",Showerthoughts,0,0,1484409994,5iq4yl,2016-12-16 19:20:42,[deleted],0
13965,liltingly,"we call basic/inferior things ""the lowest common denominator"" when the lowest common denominator of any denominators is greater than or equal to the largest",Showerthoughts,0,0,1459190542,462qd8,2016-02-16 14:47:06,NaN,0
19028,Shnezzberry,i can infer that the powerball will be soon/was recently just by all the posts on this subreddit.,Showerthoughts,0,0,1454995635,40llyi,2016-01-12 08:08:21,Am i right? I didn't check yet!,0
21745,WingmanMerlin,if my ancestors wouldn't have had sex with an inferior species i wouldn't be sneezing right now.,Showerthoughts,0,0,1455039051,4352zw,2016-01-28 20:11:11,NaN,0
23693,sammihenni,it would kinda suck to be the worlds best barber since you'd always be getting inferiors haircuts compared to your own,Showerthoughts,0,0,1454944901,3xs0ei,2015-12-22 01:30:23,NaN,0
26325,[deleted],infertile people are sterile because they can't infect anyone,Showerthoughts,0,0,1454947890,3xyf1v,2015-12-23 12:16:09,[deleted],0
28243,Ralph-Hinkley,dexy's midnight runners' biggest hit was about a woman with inferior morals and a poorly planned parlor game.,Showerthoughts,0,0,1463550611,4f2v7k,2016-04-16 17:38:00,[removed],0
28968,[deleted],"in reference to the long time period between when planet ix is inferred to exist and when it is actually discovered, we should name it something like ""higgs-bosun""",Showerthoughts,0,0,1455035723,42y3ol,2016-01-27 16:01:12,[deleted],0
29137,determinedforce,th time this week that an inference to a woman being on top sexually somehow made her more in control.,Showerthoughts,0,0,1463485824,4bakys,2016-03-21 02:47:13,"I just thought it was better for the woman to cum faster, easier.",0


In [230]:
lda.print_topics()

[(0,
  '0.129*"deputi" + 0.093*"membership" + 0.092*"appli" + 0.042*"lactas" + 0.033*"braver" + 0.033*"dispos" + 0.030*"masculinist" + 0.028*"martyr" + 0.021*"ketchup" + 0.020*"aimless"'),
 (1,
  '0.091*"filch" + 0.064*"ketchup" + 0.062*"gave" + 0.051*"fingerprint" + 0.037*"dispel" + 0.030*"delux" + 0.022*"childish" + 0.017*"fest" + 0.016*"blurri" + 0.014*"doggo"'),
 (2,
  '0.097*"ketchup" + 0.064*"lactas" + 0.050*"filch" + 0.034*"cream" + 0.031*"aimless" + 0.029*"fingerprint" + 0.026*"marajuana" + 0.023*"challah" + 0.018*"exampl" + 0.015*"copperhead"'),
 (3,
  '0.070*"burton" + 0.065*"ketchup" + 0.054*"cloth" + 0.048*"kudo" + 0.046*"manual" + 0.030*"merl" + 0.027*"concoct" + 0.027*"kingsman" + 0.026*"arachnologist" + 0.026*"financi"'),
 (4,
  '0.114*"ketchup" + 0.069*"lactas" + 0.046*"mausoleum" + 0.045*"amplitud" + 0.040*"appli" + 0.039*"filch" + 0.037*"detriment" + 0.022*"aimless" + 0.017*"constrictor" + 0.016*"kerri"'),
 (5,
  '0.148*"amber" + 0.135*"ketchup" + 0.040*"herebi" + 0.0

In [191]:
list(lda.get_document_topics(corpus))

[[(5, 0.17326587), (8, 0.13129093), (15, 0.33734262), (19, 0.25810063)],
 [(0, 0.23143989),
  (2, 0.3724656),
  (11, 0.097322114),
  (14, 0.082618676),
  (15, 0.15846139)],
 [(0, 0.33888888), (6, 0.12300421), (13, 0.3131543), (14, 0.1360637)],
 [(5, 0.11514878), (10, 0.5403457), (17, 0.20110019), (19, 0.08186689)],
 [(1, 0.08274258),
  (3, 0.113980785),
  (6, 0.3118545),
  (8, 0.082329795),
  (9, 0.096187025),
  (17, 0.18198149),
  (18, 0.080923826)],
 [(1, 0.2853853),
  (2, 0.09773442),
  (5, 0.121813215),
  (13, 0.09507277),
  (17, 0.20903987),
  (18, 0.13262106)],
 [(5, 0.3402037),
  (6, 0.27765214),
  (15, 0.10474208),
  (16, 0.09581263),
  (18, 0.11340761)],
 [(2, 0.101590894),
  (3, 0.33099663),
  (5, 0.07018978),
  (7, 0.23301001),
  (11, 0.20171268)],
 [(10, 0.3056018), (13, 0.6043982)],
 [(3, 0.53774625), (12, 0.39796805)],
 [(3, 0.3390742), (5, 0.22967248), (6, 0.14736627), (17, 0.23055372)],
 [(0, 0.42384356),
  (4, 0.052748043),
  (7, 0.26917198),
  (11, 0.06324329),
  (17,

## sklearn implementation of LDA

In [274]:
lda_sk = LatentDirichletAllocation(n_components = 20, learning_method='online', learning_decay = 0.6,
                                   learning_offset = 1024, topic_word_prior = .056)

In [ ]:
lda_sk.fit(X_short_train)

In [ ]:
short_train_features = lda

In [286]:
short_test_features = lda_sk.transform(X_short_test)

ValueError: The provided data has 6105 dimensions while the model was trained with feature size 8798.

In [279]:
short_train_features.shape

(70000, 20)

In [282]:
X_short_test.shape

(930000, 16736)

In [281]:
X_short_test.shape

(930000, 16736)

In [276]:
for topic in range(10):
    print(f"TOPIC {topic}")
    for j in np.argsort(lda_sk.components_, axis=1)[topic,:10]:
        print(cv.get_feature_names()[j])
    print()

TOPIC 0
dop
ba
gap
muggl
anecdot
disson
puriti
invas
musket
oncom

TOPIC 1
obligatori
bundl
warden
chilli
jackass
bulg
prancer
stale
swerv
di

TOPIC 2
bronz
blah
ba
dop
sixth
anxious
ozon
thriller
lite
lionel

TOPIC 3
nae
ba
dop
bronz
warrant
helpless
plow
derbi
picnic
pickup

TOPIC 4
dop
ba
pedomet
incomplet
excruci
coca
apathi
ha
cognit
brothel

TOPIC 5
dop
ba
resent
reconnect
induct
excruci
ahoy
verdict
incomplet
flour

TOPIC 6
ledger
dorm
stale
leisur
firm
dop
unslic
lampoon
coli
boo

TOPIC 7
headless
kangaroo
goff
debri
kung
genesi
virtu
deficit
vixen
squirm

TOPIC 8
rink
yogi
dop
boucher
hoy
onomatopo
packer
navel
kibbl
kamikaz

TOPIC 9
crunchi
ahoy
subtract
phlegm
suburban
seedi
inconsist
grievanc
indescrib
irrespons



In [196]:
punctuation_prime = punctuation.replace('\'', '')

In [197]:
punctuation_prime

'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~'

In [200]:
def strip_punc(s):
    return ''.join(c for c in s if c not in punctuation_prime)

In [128]:
df['quality'] = df['score'].apply(lambda x: redditutils.make_labels(x))